In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation
import logging
import datetime as dt
from datetime import datetime, timedelta
import pandas as pd
import MySQLdb
from tqdm import tqdm_notebook

In [ ]:
import matplotlib.pyplot as plt
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix

In [ ]:
class Data_import_from_DB:
    
    def __init__(self, start_date=None, end_date=None, user_id=None):
        self.start_date=start_date
        self.end_date=end_date
        self.user_id=user_id
    
    
    def get_operational_period(self):
        start = datetime.strptime(self.start_date, "%Y-%m-%d").date()
        end = datetime.strptime(self.end_date, "%Y-%m-%d").date()
        date_generated = [start + timedelta(days=x) for x in range(0, (end-start).days)]
        return date_generated
    
    def Data_import(self, date_generated, user_id=None):
        connection = MySQLdb.connect(host='103.22.220.149', port=13306, user='jaeil', password='yonsei2019!', database = 'Project_Dobrain', use_unicode =True, charset = 'utf8')
        cursor = connection.cursor()
        SQL =  "SELECT * FROM Drag_Data_by_Date_JSON WHERE Date in (%s) and id in"+user_id
        in_p=', '.join(map(lambda x: '%s', date_generated))
        SQL = SQL % in_p
        #print(SQL)
        cursor.execute(SQL, date_generated)
        data= cursor.fetchall()
        data=list(data)
        #print(data)
        df=pd.read_sql(SQL, connection, params=date_generated)
#         df.columns = data.keys()
        return df

In [ ]:
user_id_list = pd.read_csv('whole_demo_170.csv')
user_id_list = str(tuple(user_id_list.User_id.values))

In [ ]:
connector = Data_import_from_DB(start_date = '2019-05-01', end_date='2019-11-30')
date=connector.get_operational_period()
#user_id_list = "('C2B242B6-C498-49D9-B55C-7871397360E0', 'C058EFD6-1E4D-49AE-A259-6367B4BD2AA6')"
result_hospital=connector.Data_import(date, user_id_list)
result_hospital.head()

In [ ]:
final=[]
for i in tqdm_notebook(range(len(result_hospital))):
    output = result_hospital['CreationDatetime_M'][i].split()[0]
    try:
        tmp = result_hospital['CreationDatetime_M'][i].split()[1]
    except:
        print(result_hospital['CreationDatetime_M'][i], i)
    tmp=tmp.replace('.',':')
    final.append(output+' '+tmp)
result_hospital['CreationDatetime_final']=final

# 처음 플레이

In [ ]:
def first_play(result_test, level_ = None):
    from datetime import datetime
    result_test=result_test.sort_values(['CreationDatetime_final','ID'], ascending=[True, True])
    result_test['Game_num']=result_test['Index_'].map(str)+'_'+result_test['Index_X'].map(str)
    #result_test = result_test[result_test['level']==level_]
    data_dic=dict(tuple(result_test.groupby('ID')))
    print('ID의 개수 : ', len(data_dic.keys()))
    first_play_data = pd.DataFrame()    
    for m in tqdm_notebook(data_dic.keys()): # ID 하나 가져오기
        data_dic2=dict(tuple(data_dic[m].groupby('Game_num')))
        for i in data_dic2.keys(): # 게임 번호 하나 가져오기
            data_dic3=dict(tuple(data_dic2[i].groupby('CreationDatetime')))
            j = min(data_dic3.keys()) #가장 이른 날짜 가져오기
            first_play_data = first_play_data.append(data_dic3[j], ignore_index = True)
    return first_play_data

In [ ]:
result_first = first_play(result_hospital)

In [ ]:
result_first.to_csv('whole_demo_170_first_play_1213.csv')

# 라인 수

In [ ]:
result_first = pd.read_csv('whole_demo_170_first_play_1213.csv')

In [ ]:
result_first = result_first.drop(columns={'Unnamed: 0'}, axis=1)

In [ ]:
result_first = result_first.drop_duplicates()

In [ ]:
result_first['Game_num']=result_first['Index_'].map(str)+'_'+result_first['Index_X'].map(str)

In [ ]:
test1 = result_first[['ID', 'Index_', 'Index_X']].drop_duplicates()

In [ ]:
result_first = result_first.drop(columns={'Index_Y'})

In [ ]:
result_first = result_first.drop_duplicates()

In [ ]:
result_line = result_first.pivot_table(index=('ID', 'Index_', 'Index_X'), aggfunc='max', values='GameGrivenIndex')

In [ ]:
result_line.reset_index(inplace=True)

In [ ]:
result_line['GameGrivenIndex'] = pd.to_numeric(result_line['GameGrivenIndex'])

In [ ]:
result_line_cnt = result_line.pivot_table(index=('ID','Index_', 'Index_X'), aggfunc='mean', values='GameGrivenIndex')

# Fly time

In [ ]:
final=[]
for i in tqdm_notebook(range(len(result_first))):
    output = result_first['CreationDatetime_final'].iloc[i].split()[0]
    try:
        tmp = result_first['CreationDatetime_final'].iloc[i].split()[1]
    except:
        print(result_first['CreationDatetime_final'].iloc[i], i)
    tmp=tmp.replace('-',':')
    final.append(output+' '+tmp)
result_first['CreationDatetime_final']=final

In [ ]:
def first_image_flytime(result_test, level_=None):
    from datetime import datetime
    result_test=result_test.sort_values(['CreationDatetime_final','GameGrivenIndex','ID'], ascending=[True, True,True])
    result_test['Game_num']=result_test['Index_'].map(str)+'_'+result_test['Index_X'].map(str)
    result_test = result_test[result_test['Level']==level_]
    data_dic=dict(tuple(result_test.groupby('ID')))
    print('ID의 개수 : ', len(data_dic.keys()))
    ID_list, Game_num_list, level_list, Creation_Date_list, flytime_list = [] ,[], [] ,[], []
    Index_list, Index_X_list = [], []
    for m in tqdm_notebook(data_dic.keys()): # ID 하나 가져오기
        data_dic2=dict(tuple(data_dic[m].groupby('Game_num')))
        for i in data_dic2.keys(): # 게임 번호 하나 가져오기
            for k in range(len(data_dic2[i]['CreationDatetime'])-1):
 
                if data_dic2[i]['GameGrivenIndex'].iloc[k] == data_dic2[i]['GameGrivenIndex'].iloc[k+1]:
                    #continue
                    flytime = 0
                    ID_list.append(m)
                    Game_num_list.append(i)
                    Index_list.append(data_dic2[i]['Index_'].iloc[k])
                    Index_X_list.append(data_dic2[i]['Index_X'].iloc[k])
                    level_list.append(data_dic2[i]['Level'].iloc[k])
                    Creation_Date_list.append(data_dic2[i]['CreationDatetime_final'].iloc[k])
                    flytime_list.append(flytime)
                else:
                    
                    if len(data_dic2[i]['CreationDatetime_final'].iloc[k])<20:
                        data_dic2[i]['CreationDatetime_final'].iloc[k] = data_dic2[i]['CreationDatetime_final'].iloc[k] +':000'
                    if len(data_dic2[i]['CreationDatetime_final'].iloc[k+1])<20:
                        data_dic2[i]['CreationDatetime_final'].iloc[k+1] =data_dic2[i]['CreationDatetime_final'].iloc[k+1] +':000'
                    #try:
                    start = datetime.strptime(data_dic2[i]['CreationDatetime_final'].iloc[k+1], '%Y-%m-%d %H:%M:%S:%f')
                    end = datetime.strptime(data_dic2[i]['CreationDatetime_final'].iloc[k], '%Y-%m-%d %H:%M:%S:%f')
                    #except:
                        #start=0
                        #nd=0
                    #    continue
                    #    start = datetime.strptime(data_dic2[i]['CreationDatetime_final'].iloc[k+1], '%Y-%m-%d %H:%M:%S:')
                    #    end = datetime.strptime(data_dic2[i]['CreationDatetime_final'].iloc[k], '%Y-%m-%d %H:%M:%S:')
                    flytime = start - end
                    ID_list.append(m)
                    Game_num_list.append(i)
                    Index_list.append(data_dic2[i]['Index_'].iloc[k])
                    Index_X_list.append(data_dic2[i]['Index_X'].iloc[k])
                    level_list.append(data_dic2[i]['Level'].iloc[k])
                    Creation_Date_list.append(data_dic2[i]['CreationDatetime_final'].iloc[k])
                    flytime_list.append(flytime)
    flytime_df = pd.DataFrame(list(zip(ID_list, Index_list,Index_X_list, level_list, Creation_Date_list, flytime_list)),
                                  columns=['ID','Index_','Index_X', 'Level', 'CreationDatetime_final', 'Flytime'])
    return flytime_df

In [ ]:
result_first_flytime = first_image_flytime(result_first, level_='A')

In [ ]:
flytime_cal = []
for i in tqdm_notebook(range(len(result_first_flytime['Flytime']))):
    try:
        flytime_cal.append(timedelta.total_seconds(result_first_flytime['Flytime'][i]))
    except TypeError:
        flytime_cal.append(0)
result_first_flytime['Flytime_cal']=flytime_cal

In [ ]:
result_first_flytime.reset_index(inplace=True) 

In [ ]:
result_first_flytime['Game_num']=result_first_flytime['Index_'].map(str)+'_'+result_first_flytime['Index_X'].map(str)

# Line length

In [ ]:
def Line_Length(result_test, level_ = None):
    from datetime import datetime
    result_test=result_test.sort_values(['CreationDatetime_final','ID'], ascending=[True, True])
    result_test['Game_num']=result_test['Index_'].map(str)+'_'+result_test['Index_X'].map(str)
    result_test = result_test[result_test['Level']==level_]
    data_dic=dict(tuple(result_test.groupby('ID')))
    print('ID의 개수 : ', len(data_dic.keys()))
    ID_list, Game_num_list, level_list, Creation_Date_list, line_length_list = [] ,[], [] ,[], []
    Index_list, Index_X_list = [], []
    for m in tqdm_notebook(data_dic.keys()): # ID 하나 가져오기
        data_dic2=dict(tuple(data_dic[m].groupby('Game_num')))
        for i in data_dic2.keys(): # 게임 번호 하나 가져오기
            for k in range(len(data_dic2[i]['PosX'])-1):
  #              try:
                line_length_list.append(np.sqrt((data_dic2[i]['PosX'].iloc[k] - data_dic2[i]['PosX'].iloc[k+1])**2 + (data_dic2[i]['PosY'].iloc[k] - data_dic2[i]['PosY'].iloc[k+1])**2))
                ID_list.append(m)
                Game_num_list.append(i)
                Index_list.append(data_dic2[i]['Index_'].iloc[k])
                Index_X_list.append(data_dic2[i]['Index_X'].iloc[k])
                level_list.append(data_dic2[i]['Level'].iloc[k])
                Creation_Date_list.append(data_dic2[i]['CreationDatetime'].iloc[k])
#                except:
 #                   continue
    linelength_df = pd.DataFrame(list(zip(ID_list, Index_list,Index_X_list, level_list, Creation_Date_list, line_length_list)),
                                  columns=['ID','Index_','Index_X', 'Level', 'Creation Date', 'Line_Length'])
    return linelength_df

In [ ]:
result_first_linelength = Line_Length(result_first, level_ = 'A')

In [ ]:
result_first_linelength['Game_num']=result_first_linelength['Index_'].map(str)+'_'+result_first_linelength['Index_X'].map(str)

# Hight, Width 통계량

In [ ]:
result_first['PosX_ratio'] = result_first['PosX']/result_first['ScreenWidth']
result_first['PosY_ratio'] = result_first['PosY']/result_first['ScreenHeight']

In [ ]:
def hight_width_cal(result_test, level_ = None):
    from datetime import datetime
    result_test=result_test.sort_values(['CreationDatetime_final','ID'], ascending=[True, True])
    result_test['Game_num']=result_test['Index_'].map(str)+'_'+result_test['Index_X'].map(str)
    result_test = result_test[result_test['Level']==level_]
    data_dic=dict(tuple(result_test.groupby('ID')))
    print('ID의 개수 : ', len(data_dic.keys()))
    ID_list, Game_num_list, level_list, Creation_Date_list, hmax_list, hmean_list, hstd_list = [] ,[], [] ,[], [], [], []
    wmax_list, wmean_list, wstd_list = [] ,[], []
    Index_list, Index_X_list = [], []
    for m in tqdm_notebook(data_dic.keys()): # ID 하나 가져오기
        data_dic2=dict(tuple(data_dic[m].groupby('Game_num')))
        for i in data_dic2.keys(): # 게임 번호 하나 가져오기
            data_dic3=dict(tuple(data_dic2[i].groupby('CreationDatetime')))
            for k in data_dic3.keys():
                hmax_list.append(data_dic3[k]['PosY_ratio'].max())
                hmean_list.append(data_dic3[k]['PosY_ratio'].mean())
                hstd_list.append(data_dic3[k]['PosY_ratio'].std())
                wmax_list.append(data_dic3[k]['PosX_ratio'].max())
                wmean_list.append(data_dic3[k]['PosX_ratio'].mean())
                wstd_list.append(data_dic3[k]['PosX_ratio'].std())
                Index_list.append(data_dic3[k]['Index_'].iloc[0])
                Index_X_list.append(data_dic3[k]['Index_X'].iloc[0])
                ID_list.append(m)
                Game_num_list.append(i)
                level_list.append(data_dic3[k]['Level'].iloc[0])
                Creation_Date_list.append(data_dic3[k]['CreationDatetime'].iloc[0])
    height_width_df = pd.DataFrame(list(zip(ID_list,level_list, Index_list,Index_X_list, Creation_Date_list,
                                       hmax_list, hmean_list, hstd_list,wmax_list, wmean_list, wstd_list)),
                                  columns=['ID', 'Level','Index_', 'Index_X','Creation_Date', 'Height_Max', 'Height_Mean',
                                          'Height_Std', 'Width_Max', 'Width_Mean', 'Width_Std'])
    return height_width_df

In [ ]:
height_width_result_first = hight_width_cal(result_first, level_ = 'A')

# 게임 플레이 시간

In [ ]:
def play_time(result_test, level_ = None):
    from datetime import datetime
    result_test=result_test.sort_values(['CreationDatetime_final','ID'], ascending=[True, True])
    result_test['Game_num']=result_test['Index_'].map(str)+'_'+result_test['Index_X'].map(str)
    result_test = result_test[result_test['Level']==level_]
    data_dic=dict(tuple(result_test.groupby('ID')))
    print('ID의 개수 : ', len(data_dic.keys()))
    ID_list, Game_num_list, level_list, Creation_Date_list, play_time = [] ,[], [] ,[], []
    Index_list, Index_X_list = [], []
    for m in tqdm_notebook(data_dic.keys()): # ID 하나 가져오기
        data_dic2=dict(tuple(data_dic[m].groupby('Game_num')))
        for i in data_dic2.keys(): # 게임 번호 하나 가져오기
            data_dic3=dict(tuple(data_dic2[i].groupby('CreationDatetime')))
            for k in data_dic3.keys():
                try:
                    start = data_dic3[k]['CreationDatetime_final'].min()
                    end = data_dic3[k]['CreationDatetime_final'].max()
                    start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S:%f')
                    end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S:%f')
                    play_time_value = end - start
                    ID_list.append(m)
                    Game_num_list.append(i)
                    level_list.append(data_dic3[k]['Level'].iloc[0])
                    Index_list.append(data_dic3[k]['Index_'].iloc[0])
                    Index_X_list.append(data_dic3[k]['Index_X'].iloc[0])
                    Creation_Date_list.append(k)
                    play_time.append(play_time_value)
                except:
                    pass
    play_time = pd.to_timedelta(play_time)
    play_time_df = pd.DataFrame(list(zip(ID_list, Game_num_list, Index_list,Index_X_list, level_list, Creation_Date_list,
                                       play_time)),
                                  columns=['ID','Game_num','Index_','Index_X', 'Level', 'Creation_Date', 'Play_time'])
    return play_time_df
                

In [ ]:
play_time_result_first = play_time(result_first, level_ = 'A')

In [ ]:
Play_time_cal = []
for n in range(len(play_time_result_first['Play_time'])):
    Play_time_cal.append(timedelta.total_seconds(play_time_result_first['Play_time'][n]))
play_time_result_first['Play_time_cal']=Play_time_cal

# Mapping

In [ ]:
# 라인 수
result_line_cnt_final_ID = result_line.pivot_table(index=('ID','Index_', 'Index_X'), aggfunc='mean', values='GameGrivenIndex')
result_line_cnt_final_ID.reset_index(inplace=True)
result_line_cnt_final_ID = result_line_cnt_final_ID.rename(columns ={'GameGrivenIndex' : 'Line_Count'})

# Fly Time
result_first_flytime_map_final_ID = result_first_flytime.pivot_table(index=('ID','Index_', 'Index_X'), aggfunc='mean', values='Flytime_cal')
result_first_flytime_map_final_ID.reset_index(inplace=True)

# 높이, 너비 통계량
height_width_result_first_final_ID = height_width_result_first.pivot_table(index=('ID','Index_', 'Index_X'), aggfunc='mean', values=('Height_Max',
       'Height_Mean', 'Height_Std', 'Width_Max', 'Width_Mean', 'Width_Std'))
height_width_result_first_final_ID.reset_index(inplace=True)

# Play time
play_time_result_first_final_ID = play_time_result_first.pivot_table(index=('ID','Index_', 'Index_X'), aggfunc='mean', values='Play_time_cal')
play_time_result_first_final_ID.reset_index(inplace=True)

# 라인의 길이
result_first_linelength_final_ID = result_first_linelength.pivot_table(index=('ID','Index_', 'Index_X'), aggfunc='mean', values='Line_Length')
result_first_linelength_final_ID.reset_index(inplace=True)

In [ ]:
Survey_final = pd.merge(result_line_cnt_final_ID, result_first_flytime_map_final_ID)
Survey_final = pd.merge(Survey_final, height_width_result_first_final_ID)
Survey_final = pd.merge(Survey_final, play_time_result_first_final_ID)
Survey_final = pd.merge(Survey_final, result_first_linelength_final_ID)
Survey_final['State']='Abnormal'

In [ ]:
Survey_final['Game_num']=Survey_final['Index_'].map(str)+'_'+Survey_final['Index_X'].map(str)

In [ ]:
Survey_final.head(10)

# 딥러닝용 input 만들기

In [ ]:
result_first.head()

In [ ]:
result_first = result_first[result_first['Level']=='A']

In [ ]:
user_id_list = pd.read_csv('whole_demo_170.csv')

In [ ]:
user_disease = user_id_list.drop_duplicates()

In [ ]:
result_first_map = pd.merge(result_first, user_disease, left_on = 'ID', right_on = 'User_id',how='left')

In [ ]:
result_first_map_test = result_first_map.copy()

In [ ]:
final=[]
for i in tqdm_notebook(range(len(result_first_map_test))):
    output = result_first_map_test['CreationDatetime_final'][i].split()[0]
    try:
        tmp = result_first_map_test['CreationDatetime_final'][i].split()[1]
    except:
        print(result_first_map_test['CreationDatetime_final'][i], i)
    tmp=tmp.replace('-',':')
    final.append(output+' '+tmp)
result_first_map_test['CreationDatetime_final']=final

In [ ]:
result_first_map_test = result_first_map_test.sort_values(['ID','Game_num', 'CreationDatetime_final'], ascending=[True, True, True])

In [ ]:
time_diff = [1]
for _ in tqdm_notebook(range(len(result_first_map_test)-1)):
    
    if len(result_first_map_test['CreationDatetime_final'][_+1]) == 19:
        result_first_map_test['CreationDatetime_final'][_+1] = result_first_map_test['CreationDatetime_final'][_+1]+':000'
    if len(result_first_map_test['CreationDatetime_final'][_+1]) != 23:
        result_first_map_test['CreationDatetime_final'][_+1] = result_first_map_test['CreationDatetime_final'][_+1]+'0'*(23-len(result_first_map_test['CreationDatetime_final'][_+1]))
    #if result_first_map['CreationDatetime_final'][_+1][-1] ==":":
    #    result_first_map['CreationDatetime_final'][_+1] = result_first_map['CreationDatetime_final'][_+1]+'00'
    #if result_first_map['CreationDatetime_final'][_][-1] ==":":
    #    result_first_map['CreationDatetime_final'][_] = result_first_map['CreationDatetime_final'][_]+'00'
    if result_first_map_test['ID'][_] == result_first_map_test['ID'][_+1] and result_first_map_test['Game_num'][_] == result_first_map_test['Game_num'][_+1]:
        try:
            time_diff.append(datetime.strptime(result_first_map_test['CreationDatetime_final'][_+1], '%Y-%m-%d %H:%M:%S:%f')\
                         - datetime.strptime(result_first_map_test['CreationDatetime_final'][_], '%Y-%m-%d %H:%M:%S:%f'))
        except:
            time_diff.append(datetime.strptime(result_first_map_test['CreationDatetime_final'][_+1], '%Y-%m-%d %H:%M:%S')\
                         - datetime.strptime(result_first_map_test['CreationDatetime_final'][_], '%Y-%m-%d %H:%M:%S'))
    else:
        time_diff.append(0)
        

In [ ]:
len_diff_ = [0]
for _ in range(len(result_first_map_test)-1):
    len_diff_.append(np.sqrt((result_first_map_test['PosX'][_+1]-result_first_map_test['PosX'][_])**2+(result_first_map_test['PosY'][_+1]-result_first_map_test['PosY'][_])**2))

In [ ]:
result_first_map = result_first_map_test.copy()

In [ ]:
time_diff__ = pd.to_timedelta(time_diff)

In [ ]:
time_diff_a = time_diff__.total_seconds()

In [ ]:
result_first_map['time_diff'] = time_diff_a
result_first_map['len_diff'] = len_diff_

In [ ]:
result_first_map['Fly_time'] = result_first_map['time_diff']

In [ ]:
# 좌표의 play time 계산
timea_sum = []
temp = 0
for _ in tqdm_notebook(range(len(time_diff_a))):
    if time_diff_a[_] != 0:
        temp += time_diff_a[_]
        timea_sum.append(temp)
    else:
        temp=0
        timea_sum.append(temp)

In [ ]:
result_first_map['Play_time'] = timea_sum

In [ ]:
result_first_map['velocity'] = result_first_map['len_diff'] / result_first_map['time_diff']
result_first_map = result_first_map.fillna(0)

In [ ]:
import sys

In [ ]:
result_first_map.replace([np.inf, -np.inf], sys.float_info.epsilon, inplace=True)

In [ ]:
vel_diff_a = [0]
for _ in tqdm_notebook(range(len(result_first_map)-1)):
    vel_diff_a.append(result_first_map['velocity'][_+1] - result_first_map['velocity'][_])

In [ ]:
result_first_map['vel_diff'] = vel_diff_a

In [ ]:
result_first_map['accelerate'] = result_first_map['vel_diff'] / result_first_map['time_diff']
result_first_map = result_first_map.fillna(0)

In [ ]:
result_first_map.replace([np.inf, -np.inf], sys.float_info.epsilon, inplace=True)

In [ ]:
len_X_diffa = [0]
for _ in tqdm_notebook(range(len(result_first_map)-1)):
    len_X_diffa.append(result_first_map['PosX'][_+1]-result_first_map['PosX'][_])
len_Y_diffa = [0]
for _ in tqdm_notebook(range(len(result_first_map)-1)):
    len_Y_diffa.append(result_first_map['PosY'][_+1]-result_first_map['PosY'][_])

In [ ]:
result_first_map['len_X_diff'] = len_X_diffa
result_first_map['len_Y_diff'] = len_Y_diffa

In [ ]:
result_first_map['velocity_X'] = result_first_map['len_X_diff'] / result_first_map['time_diff']
result_first_map['velocity_Y'] = result_first_map['len_Y_diff'] / result_first_map['time_diff']

In [ ]:
result_first_map = result_first_map.fillna(0)
result_first_map.replace([np.inf, -np.inf], sys.float_info.epsilon, inplace=True)

In [ ]:
vel_X_diff = [0]
for _ in tqdm_notebook(range(len(result_first_map)-1)):
    vel_X_diff.append(result_first_map['velocity_X'][_+1] - result_first_map['velocity_X'][_])
vel_Y_diff = [0]
for _ in tqdm_notebook(range(len(result_first_map)-1)):
    vel_Y_diff.append(result_first_map['velocity_Y'][_+1] - result_first_map['velocity_Y'][_])

In [ ]:
result_first_map['vel_X_diff'] = vel_X_diff
result_first_map['vel_Y_diff'] = vel_Y_diff

In [ ]:
result_first_map['accelerate_X'] = result_first_map['vel_X_diff'] / result_first_map['time_diff']
result_first_map['accelerate_Y'] = result_first_map['vel_Y_diff'] / result_first_map['time_diff']

In [ ]:
result_first_map = result_first_map.fillna(0)
result_first_map.replace([np.inf, -np.inf], sys.float_info.epsilon, inplace=True)

In [ ]:
result_first_map['PosX'] = result_first_map['PosX'] / result_first_map['ScreenWidth']
result_first_map['PosY'] = result_first_map['PosY'] / result_first_map['ScreenHeight']


In [ ]:
import math, sys

In [ ]:
result_first_map = result_first_map.fillna(0)
result_first_map.replace([np.inf, -np.inf], sys.float_info.epsilon, inplace=True)

In [ ]:
result_final = result_first_map[(result_first_map['Game_num']=='1_6')].sort_values('CreationDatetime_final')[['ID','diagnosis', 'PosX','PosY','velocity','velocity_X','velocity_Y',
                                                                               'accelerate','accelerate_X','accelerate_Y','Fly_time','Play_time','month_age','sex']]

In [ ]:
result_final = result_final[result_final['ID'] !='0098878ec19d8b0114e30b43474b004d']

In [ ]:
result_final = result_final.fillna(0)

In [ ]:
result_final = result_final[result_final['diagnosis'] != 'TD']

# main input

In [ ]:
result_final = pd.get_dummies(result_final, columns=['sex'])

In [ ]:
result_final.rename(columns = {'sex_0' : 'Boy', 'sex_1' : 'Girl'}, inplace=True)

In [ ]:
main_result = result_final[['ID', 'PosX', 'PosY', 'velocity', 'velocity_X',
       'velocity_Y', 'accelerate', 'accelerate_X', 'accelerate_Y', 'Fly_time',
       'Play_time', 'diagnosis']]

In [ ]:
main_result = pd.get_dummies(main_result, columns=['diagnosis'])

In [ ]:
main_result.head()

In [ ]:
main_result['diagnosis_Other'] = main_result['diagnosis_DD'] + main_result['diagnosis_MD']

In [ ]:
main_result = main_result[['ID', 'PosX', 'PosY', 'velocity', 'velocity_X', 'velocity_Y',
       'accelerate', 'accelerate_X', 'accelerate_Y', 'Fly_time', 'Play_time',
       'diagnosis_ASD', 'diagnosis_BL', 'diagnosis_Other','diagnosis_ID']]

# sub input

In [ ]:
result_final.head()

In [ ]:
sub_result = result_final[['ID', 'month_age', 'Boy' ,'Girl']].drop_duplicates()

In [ ]:
Survey_final_map = Survey_final[Survey_final['Game_num']=='1_6']

In [ ]:
Survey_final_map = pd.merge(Survey_final_map, sub_result, how = 'inner' )

In [ ]:
Survey_final_map = Survey_final_map[['ID', 'Line_Count', 'Flytime_cal', 'Height_Max',
       'Height_Mean', 'Height_Std', 'Width_Max', 'Width_Mean', 'Width_Std',
       'Play_time_cal', 'Line_Length', 'month_age', 'Boy', 'Girl']]

In [ ]:
Survey_final_map.ID.nunique()

# CSV 파일로 출력

In [ ]:
main_result.to_csv('main_input_abnormal_1213.csv')

In [ ]:
Survey_final_map.to_csv('sub_input_abnormal_1213.csv')